# ETL

Aquí se haran las transformaciones necesarias para tener datasets para optimizar las consultas

In [188]:
import pandas as pd
import numpy as np
import ast
import json

In [189]:
df_games=pd.read_parquet(r'Datasets/steam_games_clean.parquet')
df_users=pd.read_parquet(r'Datasets/users_items_clean.parquet')
df_reviews=pd.read_parquet(r'Datasets/user_reviews_clean.parquet')

In [192]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865427 entries, 0 to 3865426
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int16  
 2   item_id           int32  
 3   playtime_forever  float16
dtypes: float16(1), int16(1), int32(1), object(1)
memory usage: 59.0+ MB


In [193]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27442 entries, 0 to 27441
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27442 non-null  object 
 1   app_name      27442 non-null  object 
 2   tags          27442 non-null  object 
 3   price         27442 non-null  float64
 4   id            27442 non-null  int32  
 5   developer     27442 non-null  object 
 6   release_year  27442 non-null  object 
dtypes: float64(1), int32(1), object(5)
memory usage: 1.4+ MB


In [194]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  int32 
 2   recommend  59305 non-null  bool  
 3   review     59305 non-null  object
dtypes: bool(1), int32(1), object(2)
memory usage: 1.2+ MB


## Análisis de sentimiento
En esta sección antes de realizar otras transformaciones se realizara el análisis de sentimientos mediante la librería **TextBlob** para simplificar el proceso. Pero esta análisis se puede realizar con otras librerías más complejas o mas elaboradas.

In [195]:
from textblob import TextBlob

#Se crea una función para devolver el sentimiento de un review pero en un rango de 0 a 2
def sentiment(review):
    blob=TextBlob(review)
    resultado=1 + round(blob.sentiment.polarity)
    return resultado

In [196]:
df_reviews['sentiment']=df_reviews['review'].apply(sentiment)

In [197]:
df_reviews.drop(columns=['review'],inplace=True)

In [198]:
df_reviews

,user_id,item_id,recommend,sentiment
0,76561197970982479,1250,True,1
1,76561197970982479,22200,True,1
2,76561197970982479,43110,True,1
3,js41637,251610,True,1
4,js41637,227300,True,1
...,...,...,...,...
59300,76561198312638244,70,True,1
59301,76561198312638244,362890,True,1
59302,LydiaMorley,273110,True,1
59303,LydiaMorley,730,True,2


## Transformación
Una vez finalizado el análisis de sentimiento vamos a comenzar a transformar y unir los datasets para que la API y el modelo de similitud del coseno puedan utilizarlos

### Dataset developers

In [199]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  int32 
 2   recommend  59305 non-null  bool  
 3   sentiment  59305 non-null  int64 
dtypes: bool(1), int32(1), int64(1), object(1)
memory usage: 1.2+ MB


In [200]:
df_item_sentiment_counts = df_reviews.groupby('item_id')['sentiment'].value_counts().unstack().reset_index()

In [201]:
df_item_sentiment_counts

sentiment,item_id,0,1,2
0,10,1.0,49.0,7.0
1,20,NaN,16.0,1.0
2,30,NaN,3.0,1.0
3,40,NaN,1.0,NaN
4,50,NaN,4.0,NaN
...,...,...,...,...
3677,521340,NaN,NaN,2.0
3678,521430,NaN,1.0,NaN
3679,521570,1.0,1.0,NaN
3680,521990,NaN,1.0,NaN


In [202]:
df_item_recommend_count=df_reviews.groupby('item_id')['recommend'].value_counts().unstack().reset_index()


In [203]:
df_counts=pd.merge(df_item_sentiment_counts,df_item_recommend_count,on='item_id')

In [204]:
df_counts.fillna(0,inplace=True)

In [205]:
df_developers=pd.merge(df_games[['price','developer','release_year','id']],df_counts,left_on='id',right_on='item_id')

In [206]:
df_developers.drop('id',axis=1,inplace=True)

In [207]:
df_developers.rename(columns={'0_x':'Negative','1_x':'Neutral',2:'Positive','False_y':'False','True_y':'True'},inplace=True)

In [208]:
df_developers = df_developers.astype({'release_year':'int16','Negative': 'int16', 'Neutral': 'int16', 'Positive': 'int16', 'False': 'int16', 'True': 'int16'})

In [209]:
df_developers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         2971 non-null   float64
 1   developer     2971 non-null   object 
 2   release_year  2971 non-null   int16  
 3   item_id       2971 non-null   int32  
 4   Negative      2971 non-null   int16  
 5   Neutral       2971 non-null   int16  
 6   Positive      2971 non-null   int16  
 7   False         2971 non-null   int16  
 8   True          2971 non-null   int16  
dtypes: float64(1), int16(6), int32(1), object(1)
memory usage: 93.0+ KB


In [210]:
df_developers.to_parquet(r'Datasets/developers.parquet')

### Datasets de users

In [399]:
df_games=pd.read_parquet(r'Datasets/steam_games_clean.parquet')
df_users=pd.read_parquet(r'Datasets/users_items_clean.parquet')
df_reviews=pd.read_parquet(r'Datasets/user_reviews_clean.parquet')

In [400]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  int32 
 2   recommend  59305 non-null  bool  
 3   review     59305 non-null  object
dtypes: bool(1), int32(1), object(2)
memory usage: 1.2+ MB


In [401]:
df1=pd.merge(df_games,df_users,left_on='id',right_on='item_id',how= 'inner')
df1.drop(columns=['id','developer','app_name','tags'],inplace=True)


In [413]:
df1

,genres,price,release_year,user_id,items_count,item_id,playtime_forever
0,"[Action, Indie, Racing]",9.99,1997,UTNerd24,188,282010,5.0
1,"[Action, Indie, Racing]",9.99,1997,I_DID_911_JUST_SAYING,154,282010,0.0
2,"[Action, Indie, Racing]",9.99,1997,76561197962104795,79,282010,0.0
3,"[Action, Indie, Racing]",9.99,1997,r3ap3r78,331,282010,0.0
4,"[Action, Indie, Racing]",9.99,1997,saint556,583,282010,13.0
...,...,...,...,...,...,...,...
3006636,[Action],9.99,2004,76561198273508956,15,80,0.0
3006637,[Action],9.99,2004,76561198282090798,5,80,0.0
3006638,[Action],9.99,2004,943525,58,80,0.0
3006639,[Action],9.99,2004,76561198283312749,49,80,9.0


In [403]:
df2=df_reviews.copy()
df2=df2.drop(columns=(['review','item_id']),axis=1)
total=df2.groupby('user_id')['recommend'].count()
positivos=df2[df2['recommend']==True].groupby('user_id')['recommend'].count()
positivos=positivos.reindex(total.index,fill_value=0)

In [404]:
valor = positivos.loc["76561197970982479"]

In [405]:
valor

3

In [406]:
porcentaje_positivos=positivos*100/total

In [407]:
df2=pd.merge(df2,porcentaje_positivos,on='user_id')

In [408]:
df2.drop_duplicates(inplace=True)

In [409]:
df2.drop(columns=['recommend_x'],inplace=True)


In [410]:
df2.rename(columns={'recommend_y':'perc_recomm'},inplace=True)

In [411]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29142 entries, 0 to 59302
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      29142 non-null  object 
 1   perc_recomm  29142 non-null  float64
dtypes: float64(1), object(1)
memory usage: 683.0+ KB


### Dataset data_ML

In [167]:
df=df_games.copy()

In [168]:
#Se eliminan las columnas que no se van a utilizar
df.drop(columns=['developer','price'],inplace=True)


In [169]:
#Para simplicar se elige el primer valor de la lista dentro de la columna genres
df['genres'] = df['genres'].apply(lambda x: x[0] if len(x) > 0 else None)

In [170]:
df['tags'] = df['tags'].apply(lambda x: x[0] if len(x) > 0 else None)

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27442 entries, 0 to 27441
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        27442 non-null  object
 1   app_name      27442 non-null  object
 2   tags          27442 non-null  object
 3   id            27442 non-null  int32 
 4   release_year  27442 non-null  object
dtypes: int32(1), object(4)
memory usage: 964.9+ KB


In [172]:
df_developers['release_year']=df_developers['release_year'].astype('int16')

In [173]:
df.to_parquet(r'Datasets/data_ML.parquet')

# Prueba de ENDPOINTS

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 

In [241]:
#Se carga el dataset de los desarrolladores
df_developers=pd.read_parquet(r'Datasets/developers.parquet')

#Se normaliza la entrada
developer='valve'
developer=developer.title()

#Se verifica que el desarrollador este en el dataset
if developer in df_developers['developer'].values:
    df_developers=df_developers[df_developers['developer']==developer]
    #Se eliminan las columnas que no se van a utilizar
    df_developers.drop(columns=['Negative','Neutral','Positive','True','False','developer'],inplace=True)
    #Total de items por cada año
    total = df_developers.groupby('release_year')['price'].count()
    #Cuenta la cantidad de items que no son gratis
    no_ceros = df_developers[df_developers['price'] != 0].groupby('release_year')['price'].count()
    no_ceros= no_ceros.reindex(total.index, fill_value=0)
    #Calacula la proporción de items gratis
    proporcion_gratis =round((1- no_ceros / total)*100,2)
    proporcion_gratis=proporcion_gratis.astype('str')+'%'
    #Doy formato a la respuesta
    data = [{'Año': year, 'Cantidad de items': total[year], 'Contenido free': proporcion_gratis[year]} for year in total.index]
    data={developer:data}  
else:
    print('No existe el desarrollador')

In [242]:
data

{'Valve': [{'Año': 1998, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 1999, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2000, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2001, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2003, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2004, 'Cantidad de items': 5, 'Contenido free': '0.0%'},
  {'Año': 2006, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2007, 'Cantidad de items': 3, 'Contenido free': '33.33%'},
  {'Año': 2008, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2009, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2010, 'Cantidad de items': 2, 'Contenido free': '50.0%'},
  {'Año': 2011, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2012, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2016, 'Cantidad de items': 1, 'Contenido free': '100.0%'}]}

In [217]:
data

{'Valve': [{'Año': 1998, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 1999, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2000, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2001, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2003, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2004, 'Cantidad de items': 5, 'Contenido free': '0.0%'},
  {'Año': 2006, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2007, 'Cantidad de items': 3, 'Contenido free': '33.33%'},
  {'Año': 2008, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2009, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2010, 'Cantidad de items': 2, 'Contenido free': '50.0%'},
  {'Año': 2011, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2012, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2016, 'Cantidad de items': 1, 'Contenido free': '100.0%'}]}

In [218]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [220]:
json.dumps(data, cls=NpEncoder, ensure_ascii=False)

'{"Valve": [{"Año": 1998, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 1999, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2000, "Cantidad de items": 2, "Contenido free": "0.0%"}, {"Año": 2001, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2003, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2004, "Cantidad de items": 5, "Contenido free": "0.0%"}, {"Año": 2006, "Cantidad de items": 2, "Contenido free": "0.0%"}, {"Año": 2007, "Cantidad de items": 3, "Contenido free": "33.33%"}, {"Año": 2008, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2009, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2010, "Cantidad de items": 2, "Contenido free": "50.0%"}, {"Año": 2011, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2012, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2016, "Cantidad de items": 1, "Contenido free": "100.0%"}]}'

def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [257]:
#Se carga el dataset de los desarrolladores
df_developers=pd.read_parquet(r'Datasets/developers.parquet')
anio=2015
if anio in df_developers['release_year'].values:
    df_developers=df_developers[df_developers['release_year']==anio]
    df_developers.drop(columns=['price','item_id','Negative','Neutral','Positive','False','release_year'],axis=1,inplace=True)
    df_developers=df_developers.groupby('developer')['True'].sum()
    df_developers=df_developers.sort_values(ascending=False)
    respuesta=[{'Puesto '+str(i+1):df_developers.index[i]} for i in range(3)]
    json.dumps(respuesta, cls=NpEncoder, ensure_ascii=False)
else:
    print('Año no encontrado')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3958770077.py, line 8)

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [269]:
#Se carga el dataset de los desarrolladores
df_developers=pd.read_parquet(r'Datasets/developers.parquet')
desarrollador="valve"
desarrollador=desarrollador.title()
desarrollador

if desarrollador in df_developers['developer'].values:
    respuesta=df_developers[df_developers['developer']==desarrollador][['Positive','Negative']].sum()
else:
    print('Desarrollador no encontrado')

In [273]:
respuesta = [f"{k} = {v}" for k, v in respuesta.items()]

In [275]:
json.dumps(respuesta, cls=NpEncoder, ensure_ascii=False)

'["Positive = 1009", "Negative = 113"]'